# Cluster Analysis

## Imports, Functions, & Settings

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from glob import glob
from zipfile import ZipFile
import shutil

# Clustering
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import make_blobs
from sklearn.cluster import DBSCAN

# Imputing missing values
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.manifold import TSNE
from sklearn.manifold import MDS

import re

import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif, mutual_info_classif, RFE

# from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture

In [ ]:
# Function to remove outliers based on IQR method
def remove_outliers(df):
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    
    # Calculate the IQR
    IQR = Q3 - Q1
    
    # Define outlier bounds (1.5 * IQR rule)
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Remove outliers
    df_no_outliers = df[~((df < lower_bound) | (df > upper_bound)).any(axis=1)]
    
    return df_no_outliers


In [ ]:
def plot_healthy_mci_dementia_kmeans_knn_dbscan(fALFF_DMN_df_scaled, eps):
    """
    This function will attempt to find 3 clusters in the input df.
    The clusters correspond to healthy, mci, and demented levels 
    of cognition.

    Parameters
    ----------
    fALFF_DMN_df_scaled : pandas dataframe
        This is the pandas dataframe containing scaled data to be plot.
    """
    kmeans = KMeans(n_clusters=3)
    kmeans.fit(fALFF_DMN_df_scaled)
    pred_label = kmeans.predict(fALFF_DMN_df_scaled)
    score = silhouette_score(fALFF_DMN_df_scaled, kmeans.labels_, random_state=42)
    print("Silhouette Score:", score)
    print(f"Predicted Labels: {pred_label}")
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(fALFF_DMN_df_scaled)

    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=pred_label, cmap='viridis')
    plt.title("KMeans Clusters (PCA Reduced Data) Default Mode Network fALFF")
    plt.show()

    # Find distances to the k-th nearest neighbor (here k = min_samples)
    min_samples = 10  # minimum number of points required to form a cluster
    k = min_samples
    neighbors = NearestNeighbors(n_neighbors=k)
    neighbors_fit = neighbors.fit(X_pca)
    distances, indices = neighbors_fit.kneighbors(X_pca)


    # Sort the distances for plotting
    distances = np.sort(distances[:, k-1], axis=0)

    # Plot the k-distance graph
    plt.plot(distances)
    plt.title('K-Distance Graph')
    plt.xlabel('Points sorted by distance')
    plt.ylabel(f'Distance to {k}-th nearest neighbor')
    # Find the optimal eps by identifying the elbow point
    # The elbow can be detected by finding the point where the distance starts to increase rapidly
    # We'll find the "elbow" by looking for the maximum change in slope
    gradient = np.diff(distances)  # Calculate the differences between consecutive distances
    second_derivative = np.diff(gradient)  # Second derivative approximates the curvature
    optimal_eps_index = np.argmax(second_derivative) + 1  # Find the index where second derivative is maximal

    # Optimal eps is the distance at this index
    optimal_eps = distances[optimal_eps_index]

    # Plot horizontal line at the optimal eps value
    plt.axhline(y=optimal_eps, color='r', linestyle='--', label=f'Optimal eps = {optimal_eps:.2f}')
    plt.legend()
    plt.show()
    print(f'Optimal eps (elbow point): {optimal_eps:.2f}')

    dbscan = DBSCAN(eps=eps, min_samples=3)  # Adjust eps and min_samples
    labels = dbscan.fit_predict(fALFF_DMN_df_scaled)
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis')
    plt.title("DBSCAN Clusters (PCA Reduced Data)")
    plt.show()

## Data Loading

In [ ]:
df_dir = "../data/dataframes/"

In [ ]:
# Extract the BOLD signal data
with ZipFile(df_dir + "BOLD.zip") as zfile:
    zfile.extractall(df_dir)
BOLD_csv_l = glob(df_dir + "BOLD/" + "*.csv", recursive=True)
BOLD_df_l = []
for csv in BOLD_csv_l:
    BOLD_df_l.append(pd.read_csv(csv))
shutil.rmtree(df_dir + "BOLD")

In [ ]:
# Load fALFF, CV, & subject names
fALFF_df = pd.read_csv(df_dir + "fALFF.csv", index_col=0)
cv_df = pd.read_csv(df_dir + "cv_df.csv", index_col=0)
p_df = pd.read_csv(df_dir + "participant_imputed_cleaned_master_df.csv")

### Viewing DataFrames

In [ ]:
p_df.head()

In [ ]:
p_df.info()

In [ ]:
fALFF_df.head()

In [ ]:
cv_df.head()

In [ ]:
len(BOLD_df_l)

In [ ]:
BOLD_df_l[0].head()

## Participant Data Analysis

In [ ]:
columns = p_df.columns

In [ ]:
p_df["moca_total"].plot()
p_df["moca_total"].mean()

In [ ]:
p_df["moca_total"].std()

In [ ]:
p_df["global"].plot()

In [ ]:
p_df["global"].mean()

In [ ]:
p_df["global"].std()

## Finding Clusters

In [ ]:
# Drop non-numeric and irrelevant columns if any
df = p_df.select_dtypes(include=[np.number])
df = df.drop(columns=['sex', 'years_of_education'])
# df = df.select_dtypes(include=[np.number])  

In [ ]:
df.describe()

### Normalizing the Data

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

In [ ]:
scaled_data_df = pd.DataFrame(scaled_data)

### Checking for Missing Values

In [ ]:
scaled_data_df.isnull().values.any()

### Impute Missing Values

In [ ]:
train_split = round(len(scaled_data_df) * .8)

In [ ]:
train_split

In [ ]:
# Split data before imputation
train, test = train_test_split(scaled_data_df, test_size=0.2, random_state=42)

In [ ]:
# Defining the pipeline
pipe = Pipeline([('imputer', KNNImputer())])

In [ ]:
# Defining the param_grid 
param_grid = {'imputer__n_neighbors': range(2, 11)}

In [ ]:
# Grid Search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_absolute_error')

In [ ]:
# Fit on Training Data
grid.fit(train)

In [ ]:
# Best n neighbors
best_n = grid.best_params_['imputer__n_neighbors']
print(f"Best n neighbors: {best_n}")

### Imputing the Missing Values

In [ ]:
# Impute using best model
best_imputer = KNNImputer(n_neighbors=best_n)

In [ ]:
scaled_data_df_imputed = pd.DataFrame(best_imputer.fit_transform(scaled_data_df))

In [ ]:
sns.heatmap(scaled_data_df.corr())

In [ ]:
sns.heatmap(scaled_data_df_imputed.corr())

In [ ]:
sns.boxplot(scaled_data_df[[5, 16]])

In [ ]:
sns.boxplot(scaled_data_df_imputed[[5, 16]])

### Cleaning & Saving Imputed Dataform

In [ ]:
unscaled_data = scaler.inverse_transform(scaled_data_df_imputed)

In [ ]:
missing_columns = [c for c in p_df.columns if c not in df.columns]

In [ ]:
unscaled_data_imputed_df = pd.DataFrame(unscaled_data, columns=df.columns)

In [ ]:
p_imputed_df = pd.concat([p_df[missing_columns], unscaled_data_imputed_df], axis=1)

In [ ]:
missing_columns

In [ ]:
p_imputed_df["handedness"] = np.array([int(re.sub(r'[^0-9.]', '',value)) for value in p_imputed_df["handedness"].values])

In [ ]:
# p_df_imputed_cleaned = p_imputed_df

In [ ]:
# p_df_imputed_cleaned.to_csv("../data/dataframes/participant_imputed_cleaned_master_df.csv", index=False)

### Applying KMeans

In [ ]:
scaled_df = pd.DataFrame(scaler.fit_transform(p_df.drop(missing_columns, axis=1)), columns=p_df.drop(missing_columns, axis=1).columns)

In [ ]:
# Apply KMeans with 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
labels = kmeans.fit_predict(scaled_df)

In [ ]:
labels

In [ ]:
X = p_df["global"].values.reshape(-1, 1)

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(scaled_df)

In [ ]:
score = silhouette_score(scaled_df, kmeans.labels_)
print("Silhouette Score:", score)

### Maximizing Silhouette Score

In [ ]:
sns.heatmap(scaled_df.corr())

In [ ]:
corr_columns_to_drop = ["executive", "attention", "memory", "global", "language", "visuospatial", "sdmt_oral", "cvlt_ld_tot"]

In [ ]:
scaled_df_uncorrelated = scaled_df.drop(corr_columns_to_drop, axis=1)

In [ ]:
sns.heatmap(scaled_df_uncorrelated.corr())

### Removing Outliers

In [ ]:
scaled_df_uncorrelated_no_outliers = scaled_df_uncorrelated

In [ ]:
scaled_df_uncorrelated_no_outliers = remove_outliers(scaled_df_uncorrelated)

In [ ]:
kmean = KMeans(n_clusters=3)
kmeans.fit(scaled_df_uncorrelated_no_outliers)
pred_label = kmeans.predict(scaled_df_uncorrelated_no_outliers)
score = silhouette_score(scaled_df_uncorrelated_no_outliers, kmeans.labels_, random_state=42)
print("Silhouette Score:", score)
print(f"Predicted Labels: {pred_label}")
pca = PCA(n_components=2)
X_pca = pca.fit_transform(scaled_df_uncorrelated_no_outliers)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=pred_label, cmap='viridis')
plt.title("KMeans Clusters (PCA Reduced Data)")
plt.show()

In [ ]:
len(pred_label)

### kNN & DBSCAN

In [ ]:
# Find distances to the k-th nearest neighbor (here k = min_samples)
min_samples = 10  # minimum number of points required to form a cluster
k = min_samples
neighbors = NearestNeighbors(n_neighbors=k)
neighbors_fit = neighbors.fit(X_pca)
distances, indices = neighbors_fit.kneighbors(X_pca)

In [ ]:

# Sort the distances for plotting
distances = np.sort(distances[:, k-1], axis=0)

# Plot the k-distance graph
plt.plot(distances)
plt.title('K-Distance Graph')
plt.xlabel('Points sorted by distance')
plt.ylabel(f'Distance to {k}-th nearest neighbor')
# Find the optimal eps by identifying the elbow point
# The elbow can be detected by finding the point where the distance starts to increase rapidly
# We'll find the "elbow" by looking for the maximum change in slope
gradient = np.diff(distances)  # Calculate the differences between consecutive distances
second_derivative = np.diff(gradient)  # Second derivative approximates the curvature
optimal_eps_index = np.argmax(second_derivative) + 1  # Find the index where second derivative is maximal

# Optimal eps is the distance at this index
optimal_eps = distances[optimal_eps_index]

# Plot horizontal line at the optimal eps value
plt.axhline(y=optimal_eps, color='r', linestyle='--', label=f'Optimal eps = {optimal_eps:.2f}')
plt.legend()
plt.show()
print(f'Optimal eps (elbow point): {optimal_eps:.2f}')

In [ ]:


dbscan = DBSCAN(eps=optimal_eps, min_samples=3)  # Adjust eps and min_samples
labels = dbscan.fit_predict(scaled_df_uncorrelated_no_outliers)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis')
plt.title("DBSCAN Clusters (PCA Reduced Data)")
plt.show()

## Examining Clusters Based on fALFF

In [ ]:
fALFF_df.columns.values

In [ ]:
DMN = ['Frontal Pole', 'Precuneous Cortex', 'Angular Gyrus', 'Cingulate Gyrus, anterior division', 'Cingulate Gyrus, posterior division', 'Parahippocampal Gyrus, anterior division', 'Parahippocampal Gyrus, posterior division', 'Frontal Orbital Cortex', 'Left Hippocampus', 'Right Hippocampus', 'Temporal Pole', 'Substantia Nigra Left', 'Substantia Nigra Right']

In [ ]:
fALFF_df.loc[:, DMN].head()

In [ ]:
fALFF_df.columns.values

In [ ]:
sns.heatmap(fALFF_df.loc[:, DMN].corr())

In [ ]:
fALFF_DMN_df_scaled = pd.DataFrame(scaler.fit_transform(fALFF_df.loc[:, DMN]), columns=fALFF_df.loc[:, DMN].columns)
# cv_df

In [ ]:
fALFF_DMN_df_scaled

In [ ]:
cv_df.isna().any()

### Imputing Missing Values in fALFF

In [ ]:
scaler = StandardScaler()

In [ ]:
fALFF_df_scaled = scaler.fit_transform(fALFF_df)

In [ ]:
# Split data before imputation
# train, test = train_test_split(fALFF_DMN_df_scaled, test_size=0.2, random_state=42)
train = fALFF_df_scaled[:train_split, :]
test = fALFF_df_scaled[train_split:, :]

# Defining the pipeline
pipe = Pipeline([('imputer', KNNImputer())])

# Defining the param_grid 
param_grid = {'imputer__n_neighbors': range(2, 11)}

# Grid Search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_absolute_error')

# Fit on Training Data
grid.fit(train)

# Best n neighbors
best_n = grid.best_params_['imputer__n_neighbors']
print(f"Best n neighbors: {best_n}")

# Impute using best model
best_imputer = KNNImputer(n_neighbors=best_n)
fALFF_df_scaled_imputed = pd.DataFrame(best_imputer.fit_transform(fALFF_df_scaled), columns=fALFF_df.columns)

In [ ]:
fALFF_df_scaled_imputed.isna().any()

### Continuing Cluster Analysis

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(fALFF_DMN_df_scaled_imputed)
pred_label = kmeans.predict(fALFF_DMN_df_scaled_imputed)
score = silhouette_score(fALFF_DMN_df_scaled_imputed, kmeans.labels_, random_state=42)
print("Silhouette Score:", score)
print(f"Predicted Labels: {pred_label}")
pca = PCA(n_components=2)
X_pca = pca.fit_transform(fALFF_DMN_df_scaled_imputed)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=pred_label, cmap='viridis')
plt.title("KMeans Clusters (PCA Reduced Data) Default Mode Network fALFF")
plt.show()

### TSNE
tsne = TSNE(perplexity=5)
X_pca_tsne = tsne.fit_transform(X_pca)

plt.scatter(X_pca_tsne[:, 0], X_pca_tsne[:, 1], c=pred_label, cmap='viridis')
plt.title("KMeans Clusters (PCA Reduced Data) Default Mode Network fALFF")
plt.show()

# print(f"PCA prev_pred_label: {prev_pred_label}\n")
print(f"TSNE pred_label: {pred_label}")



In [ ]:
# prev_pred_label = pred_label

In [ ]:
p_df

In [ ]:
len(pred_label)

In [ ]:

kmeans = KMeans(n_clusters=3)
kmeans.fit(fALFF_DMN_df_scaled_imputed)
pred_label = kmeans.predict(fALFF_DMN_df_scaled_imputed)
score = silhouette_score(fALFF_DMN_df_scaled_imputed, kmeans.labels_, random_state=42)
print("Silhouette Score:", score)
print(f"Predicted Labels: {pred_label}")
pca = PCA(n_components=2)
X_pca = pca.fit_transform(fALFF_DMN_df_scaled_imputed)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=pred_label, cmap='viridis')
plt.title("KMeans Clusters (PCA Reduced Data) Default Mode Network fALFF")
plt.show()

# Find distances to the k-th nearest neighbor (here k = min_samples)
min_samples = 10  # minimum number of points required to form a cluster
k = min_samples
neighbors = NearestNeighbors(n_neighbors=k)
neighbors_fit = neighbors.fit(X_pca)
distances, indices = neighbors_fit.kneighbors(X_pca)


# Sort the distances for plotting
distances = np.sort(distances[:, k-1], axis=0)

# Plot the k-distance graph
plt.plot(distances)
plt.title('K-Distance Graph')
plt.xlabel('Points sorted by distance')
plt.ylabel(f'Distance to {k}-th nearest neighbor')
# Find the optimal eps by identifying the elbow point
# The elbow can be detected by finding the point where the distance starts to increase rapidly
# We'll find the "elbow" by looking for the maximum change in slope
gradient = np.diff(distances)  # Calculate the differences between consecutive distances
second_derivative = np.diff(gradient)  # Second derivative approximates the curvature
optimal_eps_index = np.argmax(second_derivative) + 1  # Find the index where second derivative is maximal

# Optimal eps is the distance at this index
optimal_eps = distances[optimal_eps_index]

# Plot horizontal line at the optimal eps value
plt.axhline(y=2.0, color='r', linestyle='--', label=f'Optimal eps = {optimal_eps:.2f}')
plt.legend()
plt.show()
print(f'Optimal eps (elbow point): {optimal_eps:.2f}')

dbscan = DBSCAN(eps=2.0, min_samples=3)  # Adjust eps and min_samples
labels = dbscan.fit_predict(fALFF_DMN_df_scaled_imputed)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis')
plt.title("DBSCAN Clusters (PCA Reduced Data)")
plt.show()

In [ ]:
# Predicted Labels from the Kmeans clusters of fALFF
predicted_labels_healthy_mci_dementia = [1 ,2 ,1 ,1 ,1 ,0 ,1 ,1 ,0 ,0 ,1 ,1 ,1 ,2 ,0 ,2 ,0 ,0 ,0 ,1 ,2 ,0 ,1 ,2 ,2 ,1 ,2 ,0 ,2 ,1 ,2 ,1 ,0 ,1 ,0 ,2 ,1
 ,0 ,0 ,1 ,1 ,0 ,0 ,1 ,0 ,0 ,0 ,0 ,1 ,0 ,0 ,0 ,1 ,1 ,1 ,1 ,2 ,0]

In [ ]:
plot_healthy_mci_dementia_kmeans_knn_dbscan(fALFF_DMN_df_scaled_imputed, 2.0)

### Imputing missing values in cv

In [ ]:
cv_df_scaled = scaler.fit_transform(cv_df)

In [ ]:
cv_df_scaled = pd.DataFrame(cv_df_scaled, columns=cv_df.columns)

In [ ]:
train = cv_df_scaled.loc[:train_split, :]
test = cv_df_scaled.loc[train_split:, :]


In [ ]:
# Split data before imputation
# train, test = train_test_split(cv_df_scaled, test_size=0.2, random_state=42)
# Defining the pipeline
pipe = Pipeline([('imputer', KNNImputer())])

# Defining the param_grid 
param_grid = {'imputer__n_neighbors': range(2, 11)}

# Grid Search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='neg_mean_absolute_error')

# Fit on Training Data
grid.fit(train)

# Best n neighbors
best_n = grid.best_params_['imputer__n_neighbors']
print(f"Best n neighbors: {best_n}")

# Impute using best model
best_imputer = KNNImputer(n_neighbors=best_n)
cv_df_scaled_imputed = pd.DataFrame(best_imputer.fit_transform(cv_df_scaled), columns=cv_df_scaled.columns)

In [ ]:
cv_df_scaled_imputed.isna().any()

### Continuing Cluster Analysis

In [ ]:
plot_healthy_mci_dementia_kmeans_knn_dbscan(scaler.fit_transform(cv_df_scaled_imputed.loc[:, DMN]), 1.5)

In [ ]:
# Predicted Labels from the CV of the DMN
predicted_labels_cv_dmn = [0 ,1 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,2 ,0 ,1 ,0 ,1 ,0 ,0 ,2 ,2 ,1 ,0 ,0 ,1 ,2 ,0 ,1 ,0 ,1 ,2 ,1 ,0 ,2 ,0 ,0 ,1 ,0
 ,0 ,0 ,0 ,2 ,2 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,2 ,2 ,1 ,0]

In [ ]:
predicted_labels_healthy_mci_dementia.count(2)

In [ ]:
predicted_labels_cv_dmn.count(2)

In [ ]:
predicted_labels_healthy_mci_dementia

In [ ]:
predicted_labels_cv_dmn

In [ ]:
cv_df_scaled

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(cv_df)

In [ ]:
scaler.fit(fALFF_df)

In [ ]:
fALFF_df.head()

In [ ]:
fALFF_df.isna().any()

In [ ]:
fALFF_df_imputed = pd.DataFrame(scaler.inverse_transform(fALFF_df_scaled_imputed), columns = fALFF_df.columns)

In [ ]:
fALFF_df_imputed.isna().any()

In [ ]:
fALFF_df_imputed.head()

In [ ]:
cv_df_imputed.head()

In [ ]:
fALFF_df_imputed.to_csv("../data/dataframes/fALFF_imputed.csv", index=False)
cv_df_imputed.to_csv("../data/dataframes/cv_df_imputed.csv", index=False)

## Progress Thus Far

I have a dataset and I don't know what the ground truth for the patients is. I have three known groups, Healthy, MCI, and Demented. I have computed the fALFF and CV of the BOLD regions of the following regions from the fMRI of the patients:

['Frontal Pole', 'Insular Cortex', 'Superior Frontal Gyrus',
       'Middle Frontal Gyrus',
       'Inferior Frontal Gyrus, pars triangularis',
       'Inferior Frontal Gyrus, pars opercularis', 'Precentral Gyrus',
       'Temporal Pole', 'Superior Temporal Gyrus, anterior division',
       'Superior Temporal Gyrus, posterior division',
       'Middle Temporal Gyrus, anterior division',
       'Middle Temporal Gyrus, posterior division',
       'Middle Temporal Gyrus, temporooccipital part',
       'Inferior Temporal Gyrus, anterior division',
       'Inferior Temporal Gyrus, posterior division',
       'Inferior Temporal Gyrus, temporooccipital part',
       'Postcentral Gyrus', 'Superior Parietal Lobule',
       'Supramarginal Gyrus, anterior division',
       'Supramarginal Gyrus, posterior division', 'Angular Gyrus',
       'Lateral Occipital Cortex, superior division',
       'Lateral Occipital Cortex, inferior division',
       'Intracalcarine Cortex', 'Frontal Medial Cortex',
       'Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex)',
       'Subcallosal Cortex', 'Paracingulate Gyrus',
       'Cingulate Gyrus, anterior division',
       'Cingulate Gyrus, posterior division', 'Precuneous Cortex',
       'Cuneal Cortex', 'Frontal Orbital Cortex',
       'Parahippocampal Gyrus, anterior division',
       'Parahippocampal Gyrus, posterior division', 'Lingual Gyrus',
       'Temporal Fusiform Cortex, anterior division',
       'Temporal Fusiform Cortex, posterior division',
       'Temporal Occipital Fusiform Cortex', 'Occipital Fusiform Gyrus',
       'Frontal Opercular Cortex', 'Central Opercular Cortex',
       'Parietal Opercular Cortex', 'Planum Polare',
       "Heschl's Gyrus (includes H1 and H2)", 'Planum Temporale',
       'Supracalcarine Cortex', 'Occipital Pole',
       'Left Cerebral White Matter', 'Left Cerebral Cortex',
       'Left Lateral Ventricle', 'Left Thalamus', 'Left Caudate',
       'Left Putamen', 'Left Pallidum', 'Brain-Stem', 'Left Hippocampus',
       'Left Amygdala', 'Left Accumbens', 'Right Cerebral White Matter',
       'Right Cerebral Cortex', 'Right Lateral Ventricle',
       'Right Thalamus', 'Right Caudate', 'Right Putamen',
       'Right Pallidum', 'Right Hippocampus', 'Right Amygdala',
       'Right Accumbens']

I have data with respect to the following tests:

{
    "moca_total": {
        "Description": "total score on the Montreal Cognitive Assessment",
        "Units": "based on scoring criteria"
    },
    "drs": {
        "Description": "Mattis Dementia Rating Scale 2",
        "Units": "based on scoring criteria"
    },
    "tmt_a": {
        "Description": "Trail Making Test part A",
        "Units": "seconds"
    },
    "tmt_b": {
        "Description": "Trail Making Test part B",
        "Units": "seconds"
    },
    "bta": {
        "Description": "Brief Test of Attention",
        "Units": "based on scoring criteria"
    },
    "sdmt_oral": {
        "Description": "Symbol Digits Modality Test, oral version",
        "Units": "based on scoring criteria"
    },
    "bnt": {
        "Description": "Boston Naming Test",
        "Units": "based on scoring criteria"
    },
    "cvlt15": {
        "Description": "California Verbal Learning Test, 2nd edition, trials 1-5 total learning score",
        "Units": "based on scoring criteria"
    },
    "cvlt_ld_tot": {
        "Description": "California Verbal Learning Test, 2nd edition, long delay free recall total score",
        "Units": "based on scoring criteria"
    },
    "fas": {
        "Description": "F-A-S Verbal Phonemic Fluency test",
        "Units": "based on scoring criteria"
    },
    "jlo": {
        "Description": "Judgement of Line Orientation test",
        "Units": "based on scoring criteria"
    }
}

I also have data of the following tests:

{
    "attention": {
        "Description": "Attention cognitive composite score",
        "Units": "standardized to norm 0, variance 1"
    },
    "executive": {
        "Description": "Executive function cognitive composite score",
        "Units": "standardized to norm 0, variance 1"
    },
    "global": {
        "Description": "Global cognitive composite score",
        "Units": "standardized to norm 0, variance 1"
    },
    "language": {
        "Description": "Language cognitive composite score",
        "Units": "standardized to norm 0, variance 1"
    },
    "memory": {
        "Description": "Learning and memory cognitive composite score",
        "Units": "standardized to norm 0, variance 1"
    },
    "visuospatial": {
        "Description": "Visuospatial cognitive composite score",
        "Units": "standardized to norm 0, variance 1"
    }
}

I have attempted to identify and remove correlations in the data, examine only the regions that are associated with fALFF in the DMN (because the patients have Parkinsons MCI and Parkinsons Dementia), fit a KMeans model to the data, examine the first two principal components by looking at the pca_transformation of the kmeans transformed data, but the silhouette score of the kmeans transformed data is lower than I would like (approximately 25%). False Positives are crucial, and given the data and the specifications of the data, there is 100% certainty that the 3 clean clusters exist. How do I identify patients, given this information, that are segregated into groups of healthy cognition, Parkinson's MCI, or Parkinson's Demented?

#### Explaination of the Development of Parkinson's in the Substantia Nigra
Refined Explanation of the Process:

    Dopamine Metabolism & Hydrogen Peroxide Production:

        In substantia nigra (SNc) dopaminergic neurons, dopamine is metabolized primarily by monoamine oxidase (MAO).

        MAO converts dopamine into DOPAC (3,4-Dihydroxyphenylacetic acid), producing hydrogen peroxide (H₂O₂) as a byproduct.

        Under normal conditions, mitochondria and antioxidant enzymes like glutathione peroxidase neutralize H₂O₂.

    Mitochondrial Failure & Oxidative Stress:

        In Parkinson’s disease (PD), mitochondrial dysfunction reduces the cell’s ability to process and eliminate reactive oxygen species (ROS) (such as H₂O₂ and superoxide).

        Excess ROS damages mitochondrial DNA, lipids, and proteins, worsening cellular stress.

    Dopamine Auto-Oxidation & Neurotoxicity:

        With mitochondrial failure, excess dopamine that is not properly metabolized undergoes auto-oxidation, producing:

            Dopamine-o-quinones (DAQ)

            Neuromelanin (a polymerized form of oxidized dopamine byproducts)

            Highly reactive ROS (superoxide O₂⁻, hydroxyl radicals ●OH)

        These compounds contribute to protein aggregation (including α-synuclein misfolding), oxidative damage, and neuronal toxicity.

    Iron Accumulation & Aminochrome Formation:

        Iron accumulates abnormally in the substantia nigra of PD patients.

        Iron catalyzes dopamine oxidation, forming aminochrome, which is particularly toxic to neurons.

        Aminochrome can interact with proteins and lipids, leading to lysosomal and proteasomal dysfunction, which further promotes neurodegeneration.

    Brain Acidosis & Cell Death:

        Iron-induced dopamine oxidation is accelerated in acidic environments.

        PD patients exhibit increased brain acidity (acidosis), which promotes further oxidation and neurotoxic reactions.

        The cumulative oxidative stress triggers apoptosis (programmed cell death) via mitochondrial pathways, leading to dopaminergic neuron loss in the substantia nigra.

In [ ]:
fALFF_df.columns.values

## ANOVA Testing

In [ ]:
fALFF_df_scaled_imputed

In [ ]:
# Use K-Means to discover 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
gmm = GaussianMixture(n_components=3, random_state=42)


In [ ]:
fALFF_df_scaled_imputed.loc[:, DMN].head()

In [ ]:

kmeans.fit(fALFF_df_scaled_imputed.loc[:, DMN])

# Assign discovered clusters as target variable
# kmeans_y = df["Cluster"]


In [ ]:

gmm_y = gmm.fit_predict(fALFF_df_scaled_imputed.loc[:, DMN])
# gmm_y = df["Cluster"]



In [ ]:
kmeans.labels_

In [ ]:
X_pca = pca.fit_transform(fALFF_df_scaled_imputed.loc[:, DMN])


In [ ]:
tsne = TSNE(perplexity=5)

In [ ]:

X_pca_tsne = tsne.fit_transform(X_pca)
plt.title("KMeans Coloring of Cognition using fALFF")

plt.scatter(X_pca_tsne[:, 0], X_pca_tsne[:, 1], c=kmeans.labels_, cmap="viridis")
plt.show()


In [ ]:
plt.scatter(X_pca_tsne[:, 0], X_pca_tsne[:, 1], c=gmm_y, cmap="viridis")

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster
import seaborn as sns
import matplotlib.pyplot as plt

# Compute hierarchical linkage matrix
Z = linkage(fALFF_df_scaled_imputed, method="ward")  # "ward" minimizes intra-cluster variance

# Assign 3 clusters based on tree cut
df["Cluster"] = fcluster(Z, 3, criterion="maxclust")
y = df["Cluster"]

# Visualize dendrogram
sns.clustermap(fALFF_df_scaled_imputed, method="ward", cmap="viridis", figsize=(10, 8))
plt.show()


In [ ]:
X_pca = pca.fit_transform(cv_df_scaled_imputed.loc[:, DMN])
tsne = TSNE(perplexity=5)
X_pca_tsne = tsne.fit_transform(X_pca)

In [ ]:
plt.title("KMeans Coloring of Cognition using Coefficient of Variation")

plt.scatter(X_pca_tsne[:, 0], X_pca_tsne[:, 1], c=kmeans_cv_y, cmap="viridis")
plt.show()

### Feature Importance & ANOVA Testing

In [ ]:
anova_f, anova_p = f_classif(fALFF_df_scaled_imputed.loc[:, DMN], kmeans_y)
anova_results = pd.DataFrame({"Feature": fALFF_df_scaled_imputed.loc[:, DMN].columns, "F-Score": anova_f, "p-Value": anova_p})
anova_results = anova_results.sort_values(by="F-Score", ascending=False)
print("ANOVA Results:\n", anova_results.head(10))


In [ ]:
kmeans_cv =  KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans_cv_y = kmeans_cv.fit_predict(cv_df_scaled_imputed)


In [ ]:
anova_f, anova_p = f_classif(cv_df_scaled_imputed.loc[:, DMN], kmeans_cv_y)
anova_results = pd.DataFrame({"Feature": cv_df_scaled_imputed.loc[:, DMN].columns, "F-Score": anova_f, "p-Value": anova_p})
anova_results = anova_results.sort_values(by="F-Score", ascending=False)
print("ANOVA Results:\n", anova_results.head(10))


In [ ]:
missing_columns

In [ ]:
p_imputed_df.drop(missing_columns, axis=1)